In [1]:
import argparse
import json
import os
import sys
import traceback
from glob import glob
import math
import random
import numpy as np
from dotenv import load_dotenv
from azureml.core import Dataset, Datastore, Experiment, Run, Workspace
from azureml.core.authentication import AzureCliAuthentication
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from dotnetcore2 import runtime
runtime.version = ("18", "10", "0")

In [3]:
    #cli_auth = AzureCliAuthentication()
    #Get env variables
    
    #Create folder if not exist
    train_test_data_folder = os.path.join(os.getcwd(), 'data/tmp/train_test_data')
    os.makedirs(train_test_data_folder, exist_ok=True)

    #Connect to workspace
    ws = Workspace.from_config('config.json')

    #Get data from datastore
    datastore = Datastore(ws)
    dataset = Dataset.get_by_name(ws, name='ratings')
    dataset = dataset.to_pandas_dataframe()
    dataset

In [ ]:
# Formatting the dataset.
dataset= dataset.rename({0: "reviewer", 1: "movie", 2: "score"},axis="columns")
dataset= dataset.pivot(index="reviewer",columns="movie",values="score").fillna(0).astype(int)

i=1
for index in dataset.columns:
    while(index!=i):
        dataset.insert(i,i,[0]*6040)
        i+=1
    i+=1
dataset.add_prefix('movie')
dataset

In [ ]:
# Compile the trainig set and test set
import random
dataset_test = dataset 
dataset_train = dataset
for index, row in dataset_train.iterrows():
    non_null=row.to_numpy().nonzero()[0]
    indexes = random.sample(range(0,len(row.to_numpy().nonzero()[0])-1), 10)
    for i in indexes:
        dataset_train.iat[index-1,non_null[i]]=0

In [ ]:
datastore.upload(src_dir=train_test_data_folder, target_path='imdb_train_test',overwrite=True)
    train_test_data = Dataset.File.from_files([(datastore, 'imdb_train_test')],validate=False)
    train_test_data.register(
    workspace=ws,
    name="imdb_train_test",
    description="Processed train- and testdata for consumption by model",
    create_new_version=True)